# Verification of mortality effects

- Parameter variation `VerificationEFfects` in Anylogic
- 90 replicates per iteration or scenario

In [4]:
library(data.table)
library(xtable)
library(survival)
library(texreg)
#library(metafor)
library(ggplot2)
#library(simPH)
source("../src/utils.R")

In [5]:
# read data files
m = fread("../output/data/mortality-testing-replicates-trans.csv")
p = fread("../output/data/parameters-testing-replicates-trans.csv")
e = fread("../output/data/environment-testing-replicates-trans.csv")


In [6]:
head(p)

model_date,iteration,replicate,counties,people_per_county,fertility_adjustment,max_generation,mortality_cohort_size,income_mobility_cohort_size,measurement_cohort_window,⋯,mortality_fake_exp_coeff,smoking_rank_slope_exp_coeff,smoking_rank_slope_exp_coeff_se,smoking_income_coeff,smoking_county_income_exp_z,smoking_parent_smk_coeff,selected_generations,county_export_since_generation,recurrent_time_county_data,recurrent_heavy_computations
<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
2022-02-14 at 20:13:48 UTC,1,1,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 20:19:29 UTC,1,2,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 20:25:14 UTC,1,3,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 20:30:54 UTC,1,4,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 20:36:35 UTC,1,5,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 20:42:00 UTC,1,6,30,100,2.1,30,40,60,10,⋯,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10


In [7]:
sp = varyingParameters(p, "model_date")
parameters = names(sp)[!names(sp) %in% c("iteration", "replicate")]
print(parameters)

character(0)


In [6]:
# redefine iteration and replicate indexes
# sp[, niteration := .GRP, by = parameters]
# sp[, nreplicate := 1:.N, by = niteration]
# np = sp[, c("iteration", "replicate", "niteration", "nreplicate", parameters), with = FALSE]
# m = merge(m, np, by = c("iteration", "replicate"))
# summary(np[niteration == 3, nreplicate])
# unique(np[, c("niteration", parameters), with = FALSE])

In [10]:
mean(m[, age_death])

[1] 78.2006

# Exposure upward income mobility -> Mortality risk = 0.0, exposure until 18 years old

To explore correlation of the data produced by the ABM: 

- `total_rank_slope_exposure` and `rank_slope_exposure18`: exposure measures of place's rank-rank slopes (relative mobility)
- `income_group_mobility`: individual mobilityl, absolute value of (`income_group` - `parent_income_group`)
- `total_z_income_exposure`: exposure of standardized income

In [8]:
names(m)

[1] "iteration"                    "replicate"                   
 [3] "model_time"                   "id"                          
 [5] "fertility_control"            "parent_id"                   
 [7] "generation"                   "birthdate"                   
 [9] "age_death"                    "income_group"                
[11] "income"                       "parent_income_group"         
[13] "parent_income"                "rank"                        
[15] "parent_rank_difference"       "income_group_mobility"       
[17] "county"                       "nkids"                       
[19] "nmoves"                       "nmoves_kid"                  
[21] "smoker"                       "parent_smoker"               
[23] "county_mean_income"           "county_median_income"        
[25] "county_gini"                  "county_rank_correlation"     
[27] "county_rank_slope"            "county_rank_absolute"        
[29] "total_rank_slope_exposure"    "rank_slope_exposure18"       
[31] "total_rank_absolute_exposure" "rank_absolute_exposure18"    
[33] "total_rank_cor_exposure"      "rank_cor_exposure18"         
[35] "total_z_income_exposure"      "z_income_exposure30"

In [9]:
t = copy(m)
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, lparent_income := logIncome(parent_income)]
t[, status := 1] # there is no censoring
# t[, individual_income_mobility := abs(income_group_mobility)]
# t = t[age_death > 18]
mean(t$age_death)

[1] 78.2434

In [10]:
cor(t[, .(rank_absolute_exposure18, rank_slope_exposure18, county_rank_slope, county_rank_absolute, lincome, lparent_income,
    lcounty_income, age_death, total_z_income_exposure)])

,rank_absolute_exposure18,rank_slope_exposure18,county_rank_slope,county_rank_absolute,lincome,lparent_income,lcounty_income,age_death,total_z_income_exposure
rank_absolute_exposure18,1.00000000,-0.422589842,-0.074610120,0.38185603,0.165305459,0.198423561,0.355262707,0.032292585,0.494220754
rank_slope_exposure18,-0.42258984,1.000000000,0.105349028,-0.07604939,-0.009034334,0.008367936,-0.027376791,0.000491412,-0.005440631
county_rank_slope,-0.07461012,0.105349028,1.000000000,-0.44863911,0.018386076,0.005164088,-0.003157142,0.003132192,0.007240271
county_rank_absolute,0.38185603,-0.076049386,-0.448639106,1.00000000,0.273318812,0.122779960,0.667430614,0.036094493,0.539151895
lincome,0.16530546,-0.009034334,0.018386076,0.27331881,1.000000000,0.128581290,0.378781122,0.024293927,0.319685741
lparent_income,0.19842356,0.008367936,0.005164088,0.12277996,0.128581290,1.000000000,0.163881781,0.014222177,0.206034685
lcounty_income,0.35526271,-0.027376791,-0.003157142,0.66743061,0.378781122,0.163881781,1.000000000,0.055114929,0.818067388
age_death,0.03229258,0.000491412,0.003132192,0.03609449,0.024293927,0.014222177,0.055114929,1.000000000,0.077444006
total_z_income_exposure,0.49422075,-0.005440631,0.007240271,0.53915189,0.319685741,0.206034685,0.818067388,0.077444006,1.000000000


In [11]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18, data =t)
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t)
m3 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 +  as.factor(income_group) + lcounty_income, data = t)
m4 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4)))


                          Model 1          Model 2          Model 3          Model 4        
--------------------------------------------------------------------------------------------
rank_absolute_exposure18        -0.82 ***        -0.66 ***        -0.42 ***         0.05    
                                (0.03)           (0.03)           (0.03)           (0.03)   
as.factor(income_group)2                         -0.02 ***         0.01             0.01    
                                                 (0.00)           (0.00)           (0.00)   
as.factor(income_group)3                         -0.03 ***         0.00             0.01    
                                                 (0.00)           (0.00)           (0.00)   
as.factor(income_group)4                         -0.05 ***         0.00             0.01 *  
                                                 (0.00)           (0.00)           (0.00)   
as.factor(income_group)5                         -0.10 ***        -0.

In [16]:
m1 = coxph(Surv(age_death, status) ~ rank_slope_exposure18, data =t)
m2 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group), data = t)
m3 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) + lcounty_income, data = t)
m4 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4)))


                          Model 1      Model 2         Model 3         Model 4       
-------------------------------------------------------------------------------------
rank_slope_exposure18           0.01        -0.00           -0.02           -0.00    
                               (0.02)       (0.02)          (0.02)          (0.02)   
as.factor(income_group)2                    -0.02 ***        0.01            0.01 *  
                                            (0.00)          (0.00)          (0.00)   
as.factor(income_group)3                    -0.04 ***       -0.00            0.01    
                                            (0.00)          (0.01)          (0.00)   
as.factor(income_group)4                    -0.07 ***       -0.00            0.01    
                                            (0.00)          (0.01)          (0.01)   
as.factor(income_group)5                    -0.13 ***       -0.02 ***        0.02 ** 
                                            (0.00)   